In [1]:
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
pd.options.display.max_columns = 200
pd.options.display.max_rows = 200

# Get Data

In [3]:
df_results_orig = pd.read_excel('../data/VAC Data.xlsx', sheet_name=3)
df_mapping_orig = pd.read_excel('../data/VAC Data.xlsx', sheet_name=2)

In [4]:
df_mapping_orig.head()

,Legend SP - Spouse S - Survivor O - Orphan,Statements,CAF - member or Veteran,RCMP - member or Veteran,War Service Veteran - member,Allied Veteran - member,Family Member,still serving,planning on releasing soon,Member in receipt of a VAC disability benefit,served overseas during the Second World War or the Korean War,Member received War Veterans Allowance,has a Disability Pension,received a Disability Award,Disability Pension 1% or greater,service-related illness or injury that will likely make the transition to civilian life difficult,diagnosed medical illness or injury related to service,was medically released or will be medically released,medical release a result of a service related injury or illness,medically release within the last 120 days,released in last 5 years,"needs help with daily activities, such as dressing and personal care","Disability Pension 98% or greater (or any combination of Disability Pension, Disability Award or POW Compensation of 98% or greater)",disability results in need for ongoing care,has a significant health issue that is a result of these conditions,disability benefit results in the need for special clothing or causes excessive wear and tear on your clothing,has previously participated in rehab,currently participating in rehab,Member has successfully completed the program within the last 6 months but has been unable to find gainful employment.,Member no longer participating and has a VAC approved disability benefit considered to be permanent and severe.,"Member has been assessed as having a ""Diminished Earning Capacity.""",member or Veteran of the Regular Force,has a health problem primarily resulting from service which has made the transition from the military to civilian life difficult,requires assistance finding a job transitioning from military to civilian life,lived in Canada prior to enlistment or lived in Canada for a total of 10 years since service,requires financial assistance to meet an emergency or unexpected expense,received Career Impact Allowance,has a permanent and severe impairment related to a VAC approved disability benefit,received Exceptional Incapacity Allowance,"Member has been assessed as having a ""Diminished Earning Capacity."".1",Member has a health condition that prevents him/her from being suitably and gainfully employed,qualifys for the VAC Rehabilitation program because of health condition,Member has a household income less than the CFIS or WVA rates?,Member has released within the last two years,Reserve force Veteran or member completed at least 21 months of full time service during 24 consecutive months or Special Duty Service or emergency service within the last two years,Member receives Canadian Forces Income Supplement,Member Resides in Canada,"Member requires assistance with such things as housekeeping, grounds keeping or daily activities such as dressing and personal care",Without assistance the member would require a nursing home placement or alternative living arrangements,Member is receiving compensation for this illness or injury from the Allied Country,Member resides in a nursing home or care facility,Member has an unpaid caregiver at least 18 years of age,deemed eligible for SISIP-LTD,Member has been assessed as Totally Disabled under SISIP,Has attained the age of 65,"Severe injury after April 1, 2006",At least one of the following apply to the Member's situation as a result of this incident see list 1-8,Member finding it difficult to manage a change or transition in life,You require nursing home type care,You would like to speak with a mental health professional about a personal problem or concern,Spouse or Common-law partner,Separated Spouse or Partner,Survivor,Dependent Child,Orphan (Veteran is deceased),Parent or Extended Family,Caregiver,CAF - Family Member,RCMP - Family Member,War Service Veteran - Family Member,Allied Veteran - Family Member,Member is still serving in the CAF,Family member receiving Survivor's pension,Family member receiving 

In [5]:
df_mapping = df_mapping_orig.rename(columns={df_mapping_orig.columns[0]: "Result ID", 
                                         "Statements": "Scenario",
                                         "Unnamed: 129": "Service Type",
                                         "Unnamed: 127": "Result Name",
                                         "Unnamed: 130": "Applicant",
                                         "Unnamed: 131": "Benefit",
                                         "Unnamed: 132": "Additional Details",
                                         "Unnamed: 128": "Result Description",
                                        })
df_mapping = df_mapping.drop(df_mapping.index[0])

In [6]:
df_mapping.head()

,Result ID,Scenario,CAF - member or Veteran,RCMP - member or Veteran,War Service Veteran - member,Allied Veteran - member,Family Member,still serving,planning on releasing soon,Member in receipt of a VAC disability benefit,served overseas during the Second World War or the Korean War,Member received War Veterans Allowance,has a Disability Pension,received a Disability Award,Disability Pension 1% or greater,service-related illness or injury that will likely make the transition to civilian life difficult,diagnosed medical illness or injury related to service,was medically released or will be medically released,medical release a result of a service related injury or illness,medically release within the last 120 days,released in last 5 years,"needs help with daily activities, such as dressing and personal care","Disability Pension 98% or greater (or any combination of Disability Pension, Disability Award or POW Compensation of 98% or greater)",disability results in need for ongoing care,has a significant health issue that is a result of these conditions,disability benefit results in the need for special clothing or causes excessive wear and tear on your clothing,has previously participated in rehab,currently participating in rehab,Member has successfully completed the program within the last 6 months but has been unable to find gainful employment.,Member no longer participating and has a VAC approved disability benefit considered to be permanent and severe.,"Member has been assessed as having a ""Diminished Earning Capacity.""",member or Veteran of the Regular Force,has a health problem primarily resulting from service which has made the transition from the military to civilian life difficult,requires assistance finding a job transitioning from military to civilian life,lived in Canada prior to enlistment or lived in Canada for a total of 10 years since service,requires financial assistance to meet an emergency or unexpected expense,received Career Impact Allowance,has a permanent and severe impairment related to a VAC approved disability benefit,received Exceptional Incapacity Allowance,"Member has been assessed as having a ""Diminished Earning Capacity."".1",Member has a health condition that prevents him/her from being suitably and gainfully employed,qualifys for the VAC Rehabilitation program because of health condition,Member has a household income less than the CFIS or WVA rates?,Member has released within the last two years,Reserve force Veteran or member completed at least 21 months of full time service during 24 consecutive months or Special Duty Service or emergency service within the last two years,Member receives Canadian Forces Income Supplement,Member Resides in Canada,"Member requires assistance with such things as housekeeping, grounds keeping or daily activities such as dressing and personal care",Without assistance the member would require a nursing home placement or alternative living arrangements,Member is receiving compensation for this illness or injury from the Allied Country,Member resides in a nursing home or care facility,Member has an unpaid caregiver at least 18 years of age,deemed eligible for SISIP-LTD,Member has been assessed as Totally Disabled under SISIP,Has attained the age of 65,"Severe injury after April 1, 2006",At least one of the following apply to the Member's situation as a result of this incident see list 1-8,Member finding it difficult to manage a change or transition in life,You require nursing home type care,You would like to speak with a mental health professional about a personal problem or concern,Spouse or Common-law partner,Separated Spouse or Partner,Survivor,Dependent Child,Orphan (Veteran is deceased),Parent or Extended Family,Caregiver,CAF - Family Member,RCMP - Family Member,War Service Veteran - Family Member,Allied Veteran - Family Member,Member is still serving in the CAF,Family member receiving Survivor's pension,Family member receiving War Veterans Allowance,Family membe

# Result ID Counts

In [7]:
df = df_mapping[df_mapping['Family Member'] == 'N']
df['Result ID'].value_counts()

27                    9
19                    9
9                     5
8                     5
22                    5
29                    4
7                     4
20                    3
14                    3
18                    3
21                    3
3                     3
28                    3
6                     2
4                     2
15                    2
10                    1
5                     1
2                     1
financial benefits    1
11                    1
12                    1
13                    1
16                    1
17                    1
23                    1
24                    1
25                    1
26                    1
1                     1
Name: Result ID, dtype: int64

## Number of Unique Benefits

In [8]:
df = df_mapping[['Service Type', 'Applicant', 'Benefit']]
df = df.drop_duplicates()
df.head()

,Service Type,Applicant,Benefit
11,CAF,Member,Disability Benefits
31,War Service Veteran,Member,Disability Benefits
32,RCMP,Member,Disability Benefits
33,Allied Veteran,Member,Disability Benefits
61,CAF,Member,Family Caregiver Relief Benefit


In [9]:
df = df.set_index(['Service Type', 'Applicant'])
df['Benefit'] = 1
df.groupby(level=[0,1]).sum()

Benefit
Service Type         Applicant                            
Allied Veteran       Member                              9
                     Orphan (Veteran is deceased)        7
                     Spouse / Common-law Partner         3
                     Survivor                            5
CAF                  Caregiver                           2
                     Dependent Child                     1
                     Member                             18
                     Orphan (Veteran is deceased)       10
                     Parent / Extended Family            1
                     Spouse / Common-law Partner         6
                     Survivor                           14
RCMP                 Member                              6
                     Orphan (Veteran is deceased)        1
                     Survivor                            2
War Service Veteran  Member                              9
                     Orphan (Veteran is deceased)        6
                     Spouse / Common-law Partner         3
                     Survivor                            5

## Number of unique Result IDs

In [10]:
df = df_mapping[['Service Type', 'Applicant', 'Result ID']]
df = df.drop_duplicates()
df = df.set_index(['Service Type', 'Applicant'])
df['Result ID'] = 1
df.groupby(level=[0,1]).sum()

Result ID
Service Type         Applicant                              
Allied Veteran       Member                               11
                     Orphan (Veteran is deceased)          8
                     Spouse / Common-law Partner           3
                     Survivor                              6
CAF                  Caregiver                             2
                     Dependent Child                       1
                     Member                               24
                     Orphan (Veteran is deceased)         12
                     Parent / Extended Family              1
                     Spouse / Common-law Partner           6
                     Survivor                             16
RCMP                 Member                                7
                     Orphan (Veteran is deceased)          1
                     Survivor                              2
War Service Veteran  Member                               12
                     Orphan (Veteran is deceased)          7
                     Spouse / Common-law Partner           3
                     Survivor                              6

# Number of Result IDs

In [11]:
df = df_mapping[['Benefit', 'Result ID']]
df = df.drop_duplicates()
df['Result ID'] = 1
df = df.set_index('Benefit')
df.groupby(level=[0]).sum().sort_values(by='Result ID', ascending=False).head(10)

,Result ID
Benefit,
Disability Benefits,6
War Veterans Allowance,4
Funeral and Burial Assistance,4
Canadian Forces Income Supplement,3
Veterans Independence Program for Primary Caregivers,3
Retirement Income Security Benefit,3
Rehabilitation Services,3
Attendance Allowance,2
Vocational Assistance,2


In [12]:
df = df_mapping[['Benefit', 'Applicant', 'Result ID']]
df = df.drop_duplicates()
df['Result ID'] = 1
df = df.set_index(['Benefit', 'Applicant'])
df.groupby(level=[0, 1]).sum().sort_values(by='Result ID', ascending=False).head(10)


,,Result ID
Benefit,Applicant,
Disability Benefits,Member,5
Rehabilitation Services,Member,3
Funeral and Burial Assistance,Orphan (Veteran is deceased),2
Permanent Impairment Allowance,Member,2
Posthumous DB - Orphan,Orphan (Veteran is deceased),2
War Veterans Allowance,Member,2
Funeral and Burial Assistance,Survivor,2
Retirement Income Security Benefit,Survivor,2
Transition Interview,Member,2


# Benefits for a Given Statement

In [13]:
cols = [c for c in df_mapping.columns
        if c not in ['Scenario', 'Result Name', 'Result Description', 'Additional Details'] and 'Unnamed' not in c]
df = df_mapping[cols]
df.head()

,Result ID,CAF - member or Veteran,RCMP - member or Veteran,War Service Veteran - member,Allied Veteran - member,Family Member,still serving,planning on releasing soon,Member in receipt of a VAC disability benefit,served overseas during the Second World War or the Korean War,Member received War Veterans Allowance,has a Disability Pension,received a Disability Award,Disability Pension 1% or greater,service-related illness or injury that will likely make the transition to civilian life difficult,diagnosed medical illness or injury related to service,was medically released or will be medically released,medical release a result of a service related injury or illness,medically release within the last 120 days,released in last 5 years,"needs help with daily activities, such as dressing and personal care","Disability Pension 98% or greater (or any combination of Disability Pension, Disability Award or POW Compensation of 98% or greater)",disability results in need for ongoing care,has a significant health issue that is a result of these conditions,disability benefit results in the need for special clothing or causes excessive wear and tear on your clothing,has previously participated in rehab,currently participating in rehab,Member has successfully completed the program within the last 6 months but has been unable to find gainful employment.,Member no longer participating and has a VAC approved disability benefit considered to be permanent and severe.,"Member has been assessed as having a ""Diminished Earning Capacity.""",member or Veteran of the Regular Force,has a health problem primarily resulting from service which has made the transition from the military to civilian life difficult,requires assistance finding a job transitioning from military to civilian life,lived in Canada prior to enlistment or lived in Canada for a total of 10 years since service,requires financial assistance to meet an emergency or unexpected expense,received Career Impact Allowance,has a permanent and severe impairment related to a VAC approved disability benefit,received Exceptional Incapacity Allowance,"Member has been assessed as having a ""Diminished Earning Capacity."".1",Member has a health condition that prevents him/her from being suitably and gainfully employed,qualifys for the VAC Rehabilitation program because of health condition,Member has a household income less than the CFIS or WVA rates?,Member has released within the last two years,Reserve force Veteran or member completed at least 21 months of full time service during 24 consecutive months or Special Duty Service or emergency service within the last two years,Member receives Canadian Forces Income Supplement,Member Resides in Canada,"Member requires assistance with such things as housekeeping, grounds keeping or daily activities such as dressing and personal care",Without assistance the member would require a nursing home placement or alternative living arrangements,Member is receiving compensation for this illness or injury from the Allied Country,Member resides in a nursing home or care facility,Member has an unpaid caregiver at least 18 years of age,deemed eligible for SISIP-LTD,Member has been assessed as Totally Disabled under SISIP,Has attained the age of 65,"Severe injury after April 1, 2006",At least one of the following apply to the Member's situation as a result of this incident see list 1-8,Member finding it difficult to manage a change or transition in life,You require nursing home type care,You would like to speak with a mental health professional about a personal problem or concern,Spouse or Common-law partner,Separated Spouse or Partner,Survivor,Dependent Child,Orphan (Veteran is deceased),Parent or Extended Family,Caregiver,CAF - Family Member,RCMP - Family Member,War Service Veteran - Family Member,Allied Veteran - Family Member,Member is still serving in the CAF,Family member receiving Survivor's pension,Family member receiving War Veterans Allowance,Family member receivi

In [14]:
col = df.columns[1]
col

'CAF - member or Veteran'

In [15]:
df2 = df[['Benefit', col]][:]
df2['Benefit'] = df2['Benefit'] + ' / '
df2 = df2[df2[col] == 'Y']
df2 = df2.drop_duplicates()['Benefit']
df2.head()

11                 Disability Benefits / 
61     Family Caregiver Relief Benefit / 
71                Attendance Allowance / 
81                  Clothing Allowance / 
91    Exceptional Incapacity Allowance / 
Name: Benefit, dtype: object

# Group Service Types and Applicants

for all family = Y sum the applicant strings -> Patron types

if family = N  sum the service types => Patron types


In [16]:
df = df_mapping[['Result ID', 'Family Member', 'Service Type', 'Applicant']][:]
df = df.drop_duplicates()
df['Service Type'] = df['Service Type'] + '; '
df['Applicant'] = df['Applicant'] + '; '
df = df.set_index(['Result ID', 'Family Member'])
df.head(10)

Service Type Applicant
Result ID Family Member                                  
1         N                               CAF;   Member; 
2         N                               CAF;   Member; 
3         N              War Service Veteran ;   Member; 
          N                              RCMP;   Member; 
          N                    Allied Veteran;   Member; 
4         N              War Service Veteran ;   Member; 
          N                              RCMP;   Member; 
5         N                    Allied Veteran;   Member; 
6         N                               CAF;   Member; 
7         N                               CAF;   Member;

In [17]:
df_service_types_added = pd.DataFrame(df[['Service Type']].groupby(level=[0, 1]).sum())
df_service_types_added.head()

,,,0
Result ID,Family Member,,
1,N,Service Type,CAF;
2,N,Service Type,CAF;
3,N,Service Type,War Service Veteran ; RCMP; Allied Veteran;
4,N,Service Type,War Service Veteran ; RCMP;
5,N,Service Type,Allied Veteran;


# Parsing Result Info

In [18]:
df_results = df_results_orig[:]

In [19]:
df_results['Name'] = df_results['EN MVA'].map(lambda x: BeautifulSoup(x, "html5lib").find_all('h2')[0].get_text())

In [20]:
df_results = df_results[['ID', 'Name']].set_index('ID')

In [21]:
df_results.head(20)

,Name
ID,
1,Disability Benefits
2,Disability Benefits
3,Disability Benefits
4,Disability Benefits
5,Disability Benefits
6,Family Caregiver Relief Benefit
7,Attendance Allowance
8,Clothing Allowance
9,Exceptional Incapacity Allowance
